## Exploratory Analysis using Jupyter Notebook
For further reading, we recommend: 
- [the pandas documentation](http://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html#getting)  for information about using DataFrames
- [this blog post](https://towardsdatascience.com/introduction-to-data-visualization-in-python-89a54c97fbed) for a jumpstart into visualizations
- [the matplotlib documentation](https://matplotlib.org/users/pyplot_tutorial.html) for more info about visualizations

In [1]:
import pandas as pd

#### Loading data from our GPC bucket

In [2]:
import s3fs
s3 = s3fs.S3FileSystem(anon=True)
s3.ls('twde-datalab/raw')

s3.get('twde-datalab/raw/quito_stores_sample2016-2017.csv', 
       '../data/quito_stores_sample2016-2017.csv')

In [3]:
train = pd.read_csv('../data/quito_stores_sample2016-2017.csv')

In [6]:
train.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,city,state,cluster
0,88211471,2016-08-16,44,103520,7.0,True,Quito,Pichincha,5
1,88211472,2016-08-16,44,103665,7.0,False,Quito,Pichincha,5
2,88211473,2016-08-16,44,105574,13.0,False,Quito,Pichincha,5
3,88211474,2016-08-16,44,105575,18.0,False,Quito,Pichincha,5
4,88211475,2016-08-16,44,105577,8.0,False,Quito,Pichincha,5


#### With just this glimpse, you can start to fill out your list of assumptions, hypotheses, and questions. Some of mine are:
- Question: What is the span of dates we are provided?
- Question: How many distinct store_nbr values are there?
- Question: How many distinct item_nbr values are there?
- Hypothesis: unit_sales are always positive
- Hypothesis: onpromotion is always either True or False
- Hypothesis: city and state are always going to be Quito and Pichincha
- Hypothesis: cluster is always 5
- Question: What does cluster mean and is it important to know?
- Question: How many records does the data contain?
- Question: What other data files are available?

### Here's some examples of how to address those first questions

In [7]:
# Access an entire dataframe column like you would
# the value in a python dictionary:
# (The returned object has similar pandas built-in 
# functions, like 'head' and 'max')
data = train
print(data['date'].min())
print(data['date'].max())

2016-08-16
2017-08-15


In [8]:
# Dataframe columns also have a 'unique' method,
# which can answer several of our questions from above
data['store_nbr'].unique()

array([44, 45, 46, 47, 48, 49])

In [9]:
print(data['item_nbr'].unique())
print("There are too many item numbers to display, so let's just count them for now:")
print("\n{} different item_nbr values in our data"
          .format(len(data['item_nbr'].unique())))

[ 103520  103665  105574 ... 2011468 2011448 2123839]
There are too many item numbers to display, so let's just count them for now:

3717 different item_nbr values in our data


#### It might be helpful to know the 'shape' of our data. We could count by hand (for now) the columns, but how many rows do we have altogether?

In [10]:
print(data.shape)
print("There are {} rows and {} columns in our data".format(data.shape[0], data.shape[1]))

(5877318, 9)
There are 5877318 rows and 9 columns in our data


#### Moving along to answer our intial questions... Let's have a look at unit_sales. Keep in mind that unit sales is the variable we want to predict with our science.

Each row in our data is essentially telling us a `unit_sales` number for a given `item_nbr` at a given `store_nbr` on a given `date`. That is, "how many of an item was sold at a store on a day".

In [15]:
print(data['onpromotion'].unique())
    

[ True False]


In [16]:
[{n: len(data[data['cluster'] == n])} for n in data['cluster'].unique()]

[{5: 1014154}, {11: 1983536}, {14: 2879628}]

In [17]:
print(data['cluster'].unique())

[ 5 11 14]


In [18]:
print(data['city'].unique())

['Quito']


In [19]:
print(data['state'].unique())

['Pichincha']


In [20]:
s3.get('twde-datalab/raw/items.csv', 
       '../data/items.csv')
items = pd.read_csv('../data/items.csv')
items.sample(10)

,item_nbr,family,class,perishable
2769,1489741,PRODUCE,2010,1
4018,2116139,GROCERY I,1068,0
1515,1066901,BEVERAGES,1136,0
49,119187,CLEANING,3044,0
1726,1156724,GROCERY I,1028,0
4053,2123863,"LIQUOR,WINE,BEER",1318,0
2609,1464005,BEVERAGES,1148,0
2477,1457409,HOME CARE,3108,0
1016,812751,"LIQUOR,WINE,BEER",1318,0
3158,1920096,GROCERY I,1016,0


In [21]:
train.sample(n=6)

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,city,state,cluster
4343953,115547549,2017-05-13,47,208659,13.0,True,Quito,Pichincha,14
4014893,113409920,2017-04-23,46,877514,10.0,False,Quito,Pichincha,14
541666,91480200,2016-09-19,47,369432,1.0,False,Quito,Pichincha,14
4001698,113300407,2017-04-22,47,1471912,6.0,False,Quito,Pichincha,14
5236854,121385043,2017-07-07,46,522721,1.0,False,Quito,Pichincha,14
572117,91665237,2016-09-21,47,414354,7.0,False,Quito,Pichincha,14


In [22]:
s3.get('twde-datalab/raw/oil.csv', 
       '../data/oil.csv')
items = pd.read_csv('../data/oil.csv')
items.sample(10)

,date,dcoilwtico
694,2015-08-31,49.20
1103,2017-03-24,47.30
283,2014-01-31,97.55
691,2015-08-26,38.50
271,2014-01-15,93.78
19,2013-01-28,95.95
1092,2017-03-09,48.75
900,2016-06-14,48.49
209,2013-10-21,99.28
421,2014-08-13,97.57


In [23]:
help(pd.DataFrame.join)

Help on function join in module pandas.core.frame:

join(self, other, on=None, how='left', lsuffix='', rsuffix='', sort=False)
    Join columns of another DataFrame.
    
    Join columns with `other` DataFrame either on index or on a key
    column. Efficiently join multiple DataFrame objects by index at once by
    passing a list.
    
    Parameters
    ----------
    other : DataFrame, Series, or list of DataFrame
        Index should be similar to one of the columns in this one. If a
        Series is passed, its name attribute must be set, and that will be
        used as the column name in the resulting joined DataFrame.
    on : str, list of str, or array-like, optional
        Column or index level name(s) in the caller to join on the index
        in `other`, otherwise joins index-on-index. If multiple
        values given, the `other` DataFrame must have a MultiIndex. Can
        pass an array as the join key if it is not already contained in
        the calling DataFrame. L

In [24]:
train.join(items, on=['item_nbr'], rsuffix='_it')

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,city,state,cluster,date_it,dcoilwtico
0,88211471,2016-08-16,44,103520,7.000,True,Quito,Pichincha,5,NaN,NaN
1,88211472,2016-08-16,44,103665,7.000,False,Quito,Pichincha,5,NaN,NaN
2,88211473,2016-08-16,44,105574,13.000,False,Quito,Pichincha,5,NaN,NaN
3,88211474,2016-08-16,44,105575,18.000,False,Quito,Pichincha,5,NaN,NaN
4,88211475,2016-08-16,44,105577,8.000,False,Quito,Pichincha,5,NaN,NaN
5,88211476,2016-08-16,44,105693,8.000,False,Quito,Pichincha,5,NaN,NaN
6,88211477,2016-08-16,44,105737,7.000,False,Quito,Pichincha,5,NaN,NaN
7,88211478,2016-08-16,44,105857,27.000,False,Quito,Pichincha,5,NaN,NaN
8,88211479,2016-08-16,44,106716,5.000,False,Quito,Pichincha,5,NaN,NaN
9,88211480,2016-08-16,44,108634,2.000,False,Quito,Pichincha,5,NaN,NaN


In [28]:
train[train['perishable'] == 1]


KeyError: 'perishable'

In [29]:
train.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,city,state,cluster
0,88211471,2016-08-16,44,103520,7.0,True,Quito,Pichincha,5
1,88211472,2016-08-16,44,103665,7.0,False,Quito,Pichincha,5
2,88211473,2016-08-16,44,105574,13.0,False,Quito,Pichincha,5
3,88211474,2016-08-16,44,105575,18.0,False,Quito,Pichincha,5
4,88211475,2016-08-16,44,105577,8.0,False,Quito,Pichincha,5


In [30]:
train = train.merge(items, how='left', on='item_nbr')

KeyError: 'item_nbr'

In [31]:
train.groupby(by='family').describe()

KeyError: 'family'